In [93]:
import os
import sys
import requests
import openpyxl
import datetime
import time
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import SessionNotCreatedException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

### 동적 웹 크롤링

#### 맛집 블로그 포스트 100개 크롤링 - 제목, url, 장소명

In [94]:
# 네이버 검색 API 예제 - 블로그 검색
# post date, 
client_id = "XVCwmN8OWSwXTVWc2_uQ"
client_secret = "45zos8jb_F"
# for title in post_titles :
#     encText = urllib.parse.quote(title)

# 포스트 제목 for문으로 하나씩 넣기
encText = urllib.parse.quote("신논현 카페ㅣ강남에서 만나는 대형 뉴욕 감성 X 드렁큰빈")

url = "https://openapi.naver.com/v1/search/blog?query=" + encText # JSON 결과
display_num = "1" #출력할 갯수 입력받기
url = "https://openapi.naver.com/v1/search/blog?query=" + encText +"&display="+display_num# json 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{
	"lastBuildDate":"Mon, 05 Aug 2024 00:15:44 +0900",
	"total":371,
	"start":1,
	"display":1,
	"items":[
		{
			"title":"<b>신논현 카페ㅣ강남에서 만나는 대형 뉴욕 감성 X 드렁큰빈<\/b>",
			"link":"https:\/\/blog.naver.com\/hyobin_mo\/223533402931",
			"description":"심지어 <b>감성<\/b>이 터지는 포토 맛집이라고 하길래 후다닥 다녀온 오늘의 <b>드렁큰빈<\/b> <b>신논현 카페<\/b> <b>강남<\/b> <b>대형<\/b>... 내려가면 <b>카페<\/b>가 나올지, 진짜 지하철 역사가 나올지 궁금증을 자아낸다 지하 1층 <b>뉴욕<\/b> 지하철역 지하... ",
			"bloggername":"나와 당신, 우리만의 궤적",
			"bloggerlink":"blog.naver.com\/hyobin_mo",
			"postdate":"20240801"
		}
	]
}


In [95]:
def GetNaverBlog(url, crawler_date, crawler_time):
    App = "Naver Blog"
    data = []

    #크롬 드라이버 연결
    try:
        driver = webdriver.Chrome()
        #크롬 드라이버 설치 링크 : https:storage.googleapis.com/chrome-for-testing-public/127.0.6533.72/win64/chromedriver-win64.zip
        #압축 해제 후, driver.exe는 실행파일과 같은 위치 혹은 하위에 존재해야 함
        #사내망에선 드라이버의 물리적 위치 괄호 안에 지정해줘야 함
    except SessionNotCreatedException as e:
        print("SessionNotCreatedExceptions이 발생했습니다:", str(e))
    driver.get(url)
    time.sleep(3)
    
    # post_titles = driver.find_elements(By.CLASS_NAME,'title_post')

    #크롤링 시작
    print("-----네이버 블로그 - 맛집 포스트-----")
    for page in range(1,3): ##1~2페이지까지. 5분 소요
        driver.find_element(By.XPATH, '/html/body/ui-view/div/main/div[1]/div/section/div[2]/div[11]/span[{}]/a'.format(page)).click()
        time.sleep(2)

        #포스트 제목
        post_titles = driver.find_elements(By.CLASS_NAME,'title_post')

        #포스트 url, 식당명, 식당주소
        post_urls = driver.find_elements(By.CLASS_NAME,'desc_inner')
        urls = []
        name = []
        address = []

        for i in post_urls :
            href = i.get_attribute('href')
            # print(href)
            urls.append(href)

        for url in urls :
            res_name, res_address = GetAddress(url)
            name.append(res_name)
            address.append(res_address)

        for idx1, post_title in enumerate(post_titles[9:]): ## Top 3 제거(제목너무 짧음), 공백 행 제거
            title = post_title.text
            line_data = []
            line_data.append(App)
            line_data.append(title)
            line_data.append(urls[idx1])
            line_data.append(map(str,name[idx1]))
            line_data.append(map(str,address[idx1]))
            line_data.append(crawler_time)
            data.append(line_data) 
        
    #데이터 프레임 생성
    df = pd.DataFrame(data, columns= ['App', 'Post Title', 'URL', 'Restaurant Name', 'Restaurant Address', 'Crawler Time'])
    df.index = df.index+1
    df

    #엑셀로 저장
    df.to_excel('Results/FD_Crawling_Naver_Blog_{}.xlsx'.format(crawler_date))
    driver.close()

In [96]:
def GetAddress(url):
    print(url)
    try:
        driver = webdriver.Chrome()
        #크롬 드라이버 설치 링크 : https:storage.googleapis.com/chrome-for-testing-public/127.0.6533.72/win64/chromedriver-win64.zip
        #압축 해제 후, driver.exe는 실행파일과 같은 위치 혹은 하위에 존재해야 함
        #사내망에선 드라이버의 물리적 위치 괄호 안에 지정해줘야 함
    except SessionNotCreatedException as e:
        print("SessionNotCreatedExceptions이 발생했습니다:", str(e))
        
    #블로그 링크 하나씩 불러서 크롤링
    contents = []
    restaurant_name = []
    restaurant_address = []
    
    #블로그 링크 하나씩 불러오기
    driver.get(url)
    time.sleep(1)
    #블로그 안 본문이 있는 iframe에 접근하기
    driver.switch_to.frame("mainFrame")
    #본문 내용 크롤링하기
    #본문 내용 크롤링하기
    try:
        a = driver.find_element(By.CSS_SELECTOR,'div.se-main-container').text
        contents.append(a)
    # NoSuchElement 오류시 예외처리(구버전 블로그에 적용)
    except NoSuchElementException:
        a = driver.find_element(By.CSS_SELECTOR,'div#content-area').text
        contents.append(a)
    #print(본문: \n', a)

    # driver.quit() #창닫기
    print("<<본문 크롤링이 완료되었습니다.>>")
    # print(contents)

    print("<<본문의 장소 정보입니다.>>")
    text = contents[0]
    contents_list = text.split("\n")
    print(contents_list)
    try:
        found = contents_list.index('© NAVER Corp.') ## 네이버 지도 전체가 위젯으로 넣어진 경우
        restaurant_name.append(contents_list[found+1])
        restaurant_address.append(contents_list[found+2])
        print(restaurant_name)
        print(restaurant_address)
    except ValueError:
        try : 
            found = contents_list.index('예약')
            restaurant_name.append(contents_list[found-2])
            restaurant_address.append(contents_list[found-1])
            print(restaurant_name)
            print(restaurant_address)
        except ValueError:
            try : 
                found = contents_list.index('주소')
                restaurant_name.append(contents_list[found-1])
                restaurant_address.append(contents_list[found+1])
                print(restaurant_name)
                print(restaurant_address)
            except ValueError:
                print("장소 정보를 찾을 수 없습니다.") ## 네이버 지도 일부가 위젯으로 넣어지거나, 지도가 본문에 첨부되지 않은 경우
                restaurant_name.append(" ")
                restaurant_address.append(" ")
                print(restaurant_name)
                print(restaurant_address)

    return restaurant_name, restaurant_address
    driver.quit() #창닫기

#### 공정위 가맹사업정보제공시스템 정보공개서 크롤링 - 100개

In [97]:
def GetBusinessInfoSystem (url, crawler_date, crawler_time):
    
    try:
        driver = webdriver.Chrome()
        #크롬 드라이버 설치 링크 : https:storage.googleapis.com/chrome-for-testing-public/127.0.6533.72/win64/chromedriver-win64.zip
        #압축 해제 후, driver.exe는 실행파일과 같은 위치 혹은 하위에 존재해야 함
        #사내망에선 드라이버의 물리적 위치 괄호 안에 지정해줘야 함
    except SessionNotCreatedException as e:
        print("SessionNotCreatedExceptions이 발생했습니다:", str(e))
    driver.get(url)
    time.sleep(3)

    #외식 업종, 100개씩 보기, 적용 조회
    driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/div[2]/div[2]/form/div[1]/fieldset/select[2]/option[2]').click()
    driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/div[2]/div[2]/form/div[1]/fieldset/input[2]').click()
    driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/div[2]/div[2]/form/div[2]/div/select/option[4]').click()
    time.sleep(3)
    driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/div[2]/div[2]/form/div[2]/div/a').click() #적용 클릭
    time.sleep(3)
    table = driver.find_element(By.XPATH, '/html/body/div[2]/div[3]/div/div[2]/div[2]/form/table')
    tbody = table.find_element(By.TAG_NAME, 'tbody')

    i = 1
    data = []
    line_data = []
    
    wb = openpyxl.Workbook()
    sheet = wb.active
    sheet.append(["No.", "Name", "Business", "Representative","Registration_Number", "Init_Date","Type"])
    # 번호, 상호, 영업표지, 대표자, 등록번호, 최초등록일, 업종

    rows = tbody.find_elements(By.TAG_NAME, "tr")

    #테이블 파싱, 컬럼명 지정
    for index, value in enumerate(rows):
        Name = value.find_elements(By.TAG_NAME,"td")[1].text
        Business = value.find_elements(By.TAG_NAME,"td")[2].text
        Representative = value.find_elements(By.TAG_NAME,"td")[3].text
        Registration_Number = value.find_elements(By.TAG_NAME,"td")[4].text
        Init_Date = value.find_elements(By.TAG_NAME,"td")[5].text
        Type = value.find_elements(By.TAG_NAME,"td")[6].text

        sheet.append([i, Name, Business, Representative, Registration_Number, Init_Date, Type])
        i +=1

    for row in sheet.iter_rows(max_col=7, values_only=True):
        print(row)
        line_data.append(row)
        data.append(line_data)

    #saving to excel
    # 정렬 전 저장
    wb.save('./results/FD_Crawling_Business_Info_System_{}.xlsx'.format(crawler_date))

    # 정렬 후 저장
    xl = pd.ExcelFile('./results/FD_Crawling_Business_Info_System_{}.xlsx'.format(crawler_date))
    df = xl.parse("Sheet")
    df = df.sort_values(by="Init_Date", ascending=False)
    df.to_excel('./results/FD_Crawling_Business_Info_System_{}.xlsx'.format(crawler_date), sheet_name="가맹사업정보", columns=["No.", "Name", "Business", "Representative","Registration_Number", "Init_Date","Type"],index=False)
    driver.close()

In [98]:
def main():
    crawler_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
    crawler_date = datetime.datetime.now().strftime('%Y%m%d')
    crawler_date = datetime.datetime.now().strftime('%Y%m%d')
    print("Naver crawler_date : {}".format(crawler_time))

    naver_url = "https://section.blog.naver.com/ThemePost.naver?directoryNo=29&activeDirectorySeq=3&currentPage=1" #네이버 블로그 맛집 카테고리 메인 화면
    GetNaverBlog(naver_url, crawler_date, crawler_time)

    crawler_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M')
    crawler_date = datetime.datetime.now().strftime('%Y%m%d')
    crawler_date = datetime.datetime.now().strftime('%Y%m%d')
    print("가맹사업정보시스템 crawler_date : {}".format(crawler_time))

    business_info_system_url = "https://franchise.ftc.go.kr/mnu/00013/program/userRqst/list.do"
    GetBusinessInfoSystem(business_info_system_url, crawler_date, crawler_time)

In [99]:
if __name__ == "__main__":
    main()

Naver crawler_date : 2024-08-05 00:15
-----네이버 블로그 - 맛집 포스트-----
https://blog.naver.com/eating_review/223536046020
https://blog.naver.com/grin3/223535893599
https://blog.naver.com/eunizzang73/223536204084
https://blog.naver.com/sconesss/223536055409
https://blog.naver.com/grin3/223536214256
https://blog.naver.com/from____ming/223536192391
https://blog.naver.com/grin3/223536074648
https://blog.naver.com/goodjobtoday_/223535973903
https://blog.naver.com/ejdh90/223536389270
https://blog.naver.com/popo33311/223536388907
https://blog.naver.com/eating_review/223536046020
<<본문 크롤링이 완료되었습니다.>>
<<본문의 장소 정보입니다.>>
['노트북이나 공부하기좋은 종로 한옥카페', '아마츄어작업실청계점', '종로3가랑 종각 중간에 위치해있는데', '대화하기도좋고 노트북이나 공부하기좋은 분위기의', '한옥카페가 있더라구요, 드립커피 원두종류도많고', '커피에 진심인거같은 아마츄어작업실청계점', '소개해드릴게요✨', '📍서울 종로구 청계천로 97-8', '🕐 [월-금] 11:00 - 23:00', '[토-일] 12:00 -23:00', '📞 010-2351-0694', 'ℹ️ 무선 인터넷, 예약, 남/녀 화장실 구분,', '반려동물 동반, 포장, 단체 이용 가능', '아마츄어작업실 외관 내부', '아마츄어작업실 청계점은', '종로3가 15번출구나 종각역 4번출구로 나오셔서', '도보로 10분정도 걸으시면 나와요!', '파고다

FileNotFoundError: [Errno 2] No such file or directory: 'FD_Crawling_Business_Info_System_20240805.xlsx'